In [17]:
from laserembeddings import Laser
from os import path
import pandas as pd
from tqdm import tqdm_notebook,tqdm
import demoji
demoji.download_codes()
import numpy as np
parent_path='../Data/New_Data_15-06-2020/'


... OK (Got response in 0.89 seconds)
Writing emoji data to /home/punyajoy/.demoji/codes.json ...
... OK


In [8]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
# annotated_df=remove_duplicates_within(annotated_df)
print(len(annotated_df))

4782


In [13]:
list_text = []
lang_text = []
for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if len(row['message_text'])>4000:
        string=row['message_text'][0:2000]+row['message_text'][-2000:]
        list_text.append(demoji.replace(string,repl=""))
    else:
        list_text.append(demoji.replace(row["message_text"],repl=""))
    lang_text.append(row["language"])

100%|██████████| 4782/4782 [00:37<00:00, 128.58it/s]


### LASER

In [15]:
laser = Laser()

embed_list=[]

length_given=len(list_text)
batch_size=64
for i in tqdm(range(0,length_given,batch_size)):
    if(i+batch_size<=length_given):
        temp_text=list_text[i:i+batch_size]
        temp_lang=lang_text[i:i+batch_size]
    else:
        temp_text=list_text[i:length_given]
        temp_lang=lang_text[i:length_given]
    embeddings = laser.embed_sentences(temp_text,lang=temp_lang)  # lang is only used for tokenization
    embed_list+=list(embeddings)

100%|██████████| 75/75 [04:45<00:00,  2.31s/it]


In [44]:
X_0 = np.array(embed_list)
y_0 = np.array(annotated_df['one_fear_speech'])

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import random

In [46]:
acc=[]
macro_f1=[]
prec=[]
recall=[]
prob=[]

skf = StratifiedKFold(n_splits=10, random_state= 2020)

for train_index, test_index in skf.split(X_0, y_0):
    print("TRAIN:", train_index[0:5], "TEST:", test_index[0:5])
    X_train, X_test = X_0[train_index], X_0[test_index]
    y_train, y_test = y_0[train_index], y_0[test_index]
    class_weights = dict(zip(np.unique(y_train), compute_class_weight("balanced", np.unique(y_train),y_train)))
    print(class_weights)
    classifier= LogisticRegression(class_weight=class_weights)
    #classifier=SVC(class_weight=class_weights,kernel='rbf',probability=True)
    classifier.fit(X_train, y_train)
    y_pred=classifier.predict(X_test)
    acc.append(accuracy_score(y_test, y_pred))
    macro_f1.append(f1_score(y_test, y_pred, average='macro'))
    prec.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    prob.append(classifier.predict_proba(X_test))


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


TRAIN: [446 447 448 449 450] TEST: [0 1 2 3 4]
{0: 0.6567460317460317, 1: 2.0949367088607596}
TRAIN: [0 1 2 3 4] TEST: [446 447 448 449 450]
{0: 0.6567460317460317, 1: 2.0949367088607596}
TRAIN: [0 1 2 3 4] TEST: [870 872 873 874 875]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [1332 1334 1335 1336 1337]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [1771 1772 1773 1774 1775]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [2251 2252 2254 2255 2256]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [2757 2758 2759 2761 2762]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [3281 3282 3284 3285 3287]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [3792 3794 3796 3799 3801]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [4302 4305 4306 4307 4308]
{0: 0.6568986568986569, 1: 2.093385214007782}


### For SVC (with rbf kernel)

In [24]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

Accuracy: 0.68 (+/- 0.11)
Macro F1: 0.63 (+/- 0.08)
Precision for +ve class: 0.40 (+/- 0.14)
Recall for +ve class: 0.64 (+/- 0.12)


### For logisitic regression

In [47]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

Accuracy: 0.67 (+/- 0.12)
Macro F1: 0.62 (+/- 0.10)
Precision for +ve class: 0.40 (+/- 0.15)
Recall for +ve class: 0.67 (+/- 0.11)


### Doc2Vec


In [32]:
from utils.preprocess import *
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

list_sents=[preprocess_sent(ele,params={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': True}) for ele in tqdm(annotated_df['message_text'],total=len(annotated_df))]
X_0 = np.array(list_sents,dtype='object')
y_0 = np.array(annotated_df['one_fear_speech'])

100%|██████████| 4782/4782 [00:57<00:00, 83.43it/s] 


In [41]:
acc=[]
macro_f1=[]
prec=[]
recall=[]
prob=[]

skf = StratifiedKFold(n_splits=10, random_state= 2020)

for train_index, test_index in skf.split(X_0, y_0):
    print("TRAIN:", train_index[0:5], "TEST:", test_index[0:5])
    X_train, X_test = X_0[train_index], X_0[test_index]
    y_train, y_test = y_0[train_index], y_0[test_index]
    class_weights = dict(zip(np.unique(y_train), compute_class_weight("balanced", np.unique(y_train),y_train)))
    
    
    print(class_weights)
    
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(X_train)]
    model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=10)
    X_train_embed = np.array([list(model.infer_vector(ele)) for ele in X_train])
    X_test_embed = np.array([list(model.infer_vector(ele)) for ele in X_test])
    #classifier= LogisticRegression(class_weight=class_weights,max_iter=500)
    classifier=SVC(class_weight=class_weights,kernel='rbf',probability=True)
    classifier.fit(X_train_embed, y_train)
    y_pred=classifier.predict(X_test_embed)
    acc.append(accuracy_score(y_test, y_pred))
    macro_f1.append(f1_score(y_test, y_pred, average='macro'))
    prec.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    prob.append(classifier.predict_proba(X_test_embed))


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


TRAIN: [446 447 448 449 450] TEST: [0 1 2 3 4]
{0: 0.6567460317460317, 1: 2.0949367088607596}
TRAIN: [0 1 2 3 4] TEST: [446 447 448 449 450]
{0: 0.6567460317460317, 1: 2.0949367088607596}
TRAIN: [0 1 2 3 4] TEST: [870 872 873 874 875]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [1332 1334 1335 1336 1337]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [1771 1772 1773 1774 1775]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [2251 2252 2254 2255 2256]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [2757 2758 2759 2761 2762]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [3281 3282 3284 3285 3287]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [3792 3794 3796 3799 3801]
{0: 0.6568986568986569, 1: 2.093385214007782}
TRAIN: [0 1 2 3 4] TEST: [4302 4305 4306 4307 4308]
{0: 0.6568986568986569, 1: 2.093385214007782}


### SVC


In [42]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

Accuracy: 0.76 (+/- 0.06)
Macro F1: 0.70 (+/- 0.06)
Precision for +ve class: 0.52 (+/- 0.14)
Recall for +ve class: 0.60 (+/- 0.15)


### Logistic Regression

In [40]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

Accuracy: 0.74 (+/- 0.08)
Macro F1: 0.67 (+/- 0.07)
Precision for +ve class: 0.47 (+/- 0.13)
Recall for +ve class: 0.62 (+/- 0.14)


In [31]:
documents

[TaggedDocument(words=['human', 'interface', 'computer'], tags=[0]),
 TaggedDocument(words=['survey', 'user', 'computer', 'system', 'response', 'time'], tags=[1]),
 TaggedDocument(words=['eps', 'user', 'interface', 'system'], tags=[2]),
 TaggedDocument(words=['system', 'human', 'system', 'eps'], tags=[3]),
 TaggedDocument(words=['user', 'response', 'time'], tags=[4]),
 TaggedDocument(words=['trees'], tags=[5]),
 TaggedDocument(words=['graph', 'trees'], tags=[6]),
 TaggedDocument(words=['graph', 'minors', 'trees'], tags=[7]),
 TaggedDocument(words=['graph', 'minors', 'survey'], tags=[8])]

[-0.054082505, 0.09554306, -0.02666436, -0.04751123, -0.0072117187]